In [ ]:
#CNN on IDRiD Dataset

In [ ]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, optimizers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

base_path = "/content/drive/MyDrive/Grading_IDRiD"

train_dir = os.path.join(base_path, "1. Original Images", "a. Training Set")
test_dir  = os.path.join(base_path, "1. Original Images", "b. Testing Set")

train_csv = os.path.join(base_path, "2. Groundtruths", "a. IDRiD_Disease Grading_Training Labels.csv")
test_csv  = os.path.join(base_path, "2. Groundtruths", "b. IDRiD_Disease Grading_Testing Labels.csv")


In [ ]:
import pandas as pd
import numpy as np
import cv2
import os

IMG_SIZE = 128

def load_data(image_dir, label_csv):
    df = pd.read_csv(label_csv)
    df.columns = df.columns.str.strip()

    X, y = [], []

    for _, row in df.iterrows():
        img_name = row['Image name'].strip() + ".jpg"
        label = int(row['Retinopathy grade'])

        path = os.path.join(image_dir, img_name)
        img = cv2.imread(path)

        if img is not None:
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            X.append(img / 255.0)
            y.append(label)

    return np.array(X), np.array(y)

In [ ]:
x_train, y_train = load_data(train_dir, train_csv)
x_test, y_test = load_data(test_dir, test_csv)

print("Train:", x_train.shape, y_train.shape)
print("Test: ", x_test.shape, y_test.shape)


Train: (413, 128, 128, 3) (413,)
Test:  (103, 128, 128, 3) (103,)


In [ ]:
from tensorflow.keras.utils import to_categorical

NUM_CLASSES = 5  # DR Grades are from 0 to 4

y_train_cat = to_categorical(y_train, NUM_CLASSES)
y_test_cat = to_categorical(y_test, NUM_CLASSES)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Augmentation for training
train_aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    fill_mode="nearest"
)

# No augmentation for testing
test_aug = ImageDataGenerator()

# Define batch size
BATCH_SIZE = 32

# Create augmented generators
train_gen = train_aug.flow(x_train, y_train_cat, batch_size=BATCH_SIZE)
test_gen = test_aug.flow(x_test, y_test_cat, batch_size=BATCH_SIZE)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(NUM_CLASSES, activation='softmax')  # Classification layer
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# ✅ Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# ✅ Train the model using the augmented generators
history = model.fit(
    train_gen,
    steps_per_epoch=len(x_train) // BATCH_SIZE,
    validation_data=test_gen,
    validation_steps=len(x_test) // BATCH_SIZE,
    epochs=20
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 14s 986ms/step - accuracy: 0.2701 - loss: 2.0688 - val_accuracy: 0.3750 - val_loss: 1.5288
Epoch 2/20
 1/12 ━━━━━━━━━━━━━━━━━━━━ 6s 546ms/step - accuracy: 0.3448 - loss: 1.5188

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - accuracy: 0.3448 - loss: 1.5188 - val_accuracy: 0.3438 - val_loss: 1.5263
Epoch 3/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.4322 - loss: 1.4676 - val_accuracy: 0.3333 - val_loss: 1.4455
Epoch 4/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - accuracy: 0.4688 - loss: 1.2961 - val_accuracy: 0.3125 - val_loss: 1.5012
Epoch 5/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.4596 - loss: 1.4106 - val_accuracy: 0.3750 - val_loss: 1.4589
Epoch 6/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - accuracy: 0.5938 - loss: 1.2628 - val_accuracy: 0.3542 - val_loss: 1.4656
Epoch 7/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 12s 971ms/step - accuracy: 0.4420 - loss: 1.4026 - val_accuracy: 0.3854 - val_loss: 1.4688
Epoch 8/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - accuracy: 0.3438 - loss: 1.3774 - val_accuracy: 0.3854 - val_loss: 1.4554
Epoch 9/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.4218 - loss: 1.3872 - val_accuracy: 0.3750 - val_loss: 1.4

In [ ]:
loss, acc = model.evaluate(x_test, y_test_cat)
print(f"Test Accuracy: {acc:.4f}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 145ms/step - accuracy: 0.3337 - loss: 1.4691
Test Accuracy: 0.3786


In [ ]:
model.save("/content/drive/MyDrive/grading_model.h5")

ValueError: Invalid filepath extension for saving. Please add either a `.keras` extension for the native Keras format (recommended) or a `.h5` extension. Use `model.export(filepath)` if you want to export a SavedModel for use with TFLite/TFServing/etc. Received: filepath=Grading_CNN_Model.